In [1]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

import matplotlib.pyplot as plt

from common.multiprocessing_env import SubprocVecEnv

In [2]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [3]:

num_envs = 16
env_name = "Pong-v0"

In [4]:

def make_env():
    def _thunk():
        env = gym.make(env_name)
        return env

    return _thunk

envs = [make_env() for i in range(num_envs)]
envs = SubprocVecEnv(envs)

env = gym.make(env_name)

In [5]:

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class CNNBase(nn.Module):
    def __init__(self, num_inputs, num_outputs,  hidden_size=512):
        super().__init__()

        # init_ = lambda m: init(m,
        #     nn.init.orthogonal_,
        #     lambda x: nn.init.constant_(x, 0),
        #     nn.init.calculate_gain('relu'))

        self.main = nn.Sequential(
            nn.Conv2d(num_inputs, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, stride=1),
            nn.ReLU(),
            Flatten(),
            nn.Linear(32 * 7 * 7, hidden_size),
            nn.ReLU()
        )

        # init_ = lambda m: init(m,
        #     nn.init.orthogonal_,
        #     lambda x: nn.init.constant_(x, 0))

        self.critic_linear = nn.Linear(hidden_size, num_outputs)

        self.train() # eval 의 반대모드

    def forward(self, inputs, rnn_hxs, ):
        x = self.main(inputs / 255.0)
    
        return self.critic_linear(x), x#return q_value, feature

In [6]:
class ActorCritic(nn.Module):
    def __init__(self, obs_shape, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.base = CNNBase(obs_shape[0], num_outputs)

    
    def actor(self, x, hidden_size = 32):
        #todo : num_inputs 하드코딩 해결
        num_inputs = 512
        x = nn.Linear(num_inputs, hidden_size)(x)
        x = nn.LeakyReLU()(x)
        x = nn.Linear(hidden_size, num_outputs)(x)
        return x
        
    def forward(self, x):
        q_value, actor_features = self.base(x, num_outputs)
        policy    = self.actor(actor_features)
        # std   = self.log_std.exp().expand_as(mu)
        # dist  = Normal(mu, std)
        return policy, q_value

In [7]:
def plot(frame_idx, rewards):
    # clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

In [8]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage): # 전체 데이터 셋에서 mini_batch 를 만드는 것이다.
    batch_size = states.size(0)
    for _ in range(batch_size // mini_batch_size):
        rand_ids = np.random.randint(0, batch_size, mini_batch_size)
        yield states[rand_ids, :], actions[rand_ids, :], log_probs[rand_ids, :], returns[rand_ids, :], advantage[rand_ids, :]
        
        

def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2): # training
    for _ in range(ppo_epochs):
        for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
            pi, value = model(state)
            pi_a = pi.gather(1,a)
            new_log_probs = torch.log(pi_a)

            ratio = (new_log_probs - old_log_probs).exp()
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss  = - torch.min(surr1, surr2).mean()
            critic_loss = (return_ - value).pow(2).mean()

            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [9]:
num_inputs  = envs.observation_space.shape
num_outputs = envs.action_space.n

#Hyper params:
hidden_size      = 32
lr               = 1e-3
num_steps        = 128
mini_batch_size  = 256
ppo_epochs       = 30


model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device) #return dist, v
optimizer = optim.Adam(model.parameters(), lr=lr)

max_frames = 150000
frame_idx  = 0
test_rewards = []
early_stop = False

state = envs.reset()


In [10]:
while frame_idx < max_frames and not early_stop: #todo 15000 epoch 반복 -> ppo epochs 는 따로 있는데>??

    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps): #경험 모으기 - gpu 쓰는구나 . 하지만 여전히 DataParallel 들어갈 여지는 없어보인다. 
        #-> 아.. state 가 벡터 1개가 아닐 것 같다.. 16개네. gpu 쓸만하다. DataParallel 도 가능할듯?
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)

        action = dist.sample()
        next_state, reward, done, _ = envs.step(action.cpu().numpy()) #done 한다고 끝내질 않네??
        

        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        
        states.append(state)
        actions.append(action)
        
        state = next_state
        frame_idx += 1
        
        if frame_idx % 1000 == 0 and frame_idx >15000: # 1000번 마다 plot 그려주기
#             torch.save(model.state_dict(),'/home/jeffrey/RL_study/RL-Adventure-2/weight/{}.pt'.format(frame_idx))

            test_reward = np.mean([test_env() for _ in range(10)])
            test_rewards.append(test_reward)
            # plot(frame_idx, test_rewards)
            print(test_reward)
            if test_reward > threshold_reward: early_stop = True
            

    next_state = torch.FloatTensor(next_state).to(device) #경험 1개 모은거로 학습하기
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    actions   = torch.cat(actions)
    advantage = returns - values
    
    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)

RuntimeError: cuDNN error: CUDNN_STATUS_BAD_PARAM

In [12]:
state.shape

torch.Size([16, 210, 160, 3])